In [1]:
# YOLO object detection
import cv2 as cv
import numpy as np
import os

In [18]:
# Give the configuration and weight files for the model and load the network.
configPath = 'Data/YOLO/yolov4.cfg'
weightsPath = 'Data/Yolo/yolov4.weights'

net = cv.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)

# determine the output layer
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [21]:
def get_cropped_image(image, input_path, out_path):    
    # read in image and construct a blob from the image
    img = cv.imread(input_path + image)
    blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)

    # get detections
    net.setInput(blob)
    outputs = net.forward(ln)

    # initialize lists
    boxes = []
    confidences = []
    # initialize image dimensions
    h_img, w_img = img.shape[:2]

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # Only keep detection if it is for a person with high confidence
            if confidence > 0.9 and classID == 0:
                box = detection[:4] * np.array([w_img, h_img, w_img, h_img])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                box = [x, y, int(width), int(height)]
                boxes.append(box)
                confidences.append(float(confidence))

    indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            
            # check that all dimensions of bounding box are within image
            upper = y if y > 0 else 0
            lower = y+h if y+h < h_img else h_img
            left = x if x > 0 else 0
            right = x+w if x+w < w_img else w_img
            # crop image and save
            crop_img = img[upper:lower, left:right]
            cv.imwrite(out_path+image[:-4]+'_'+'crop_'+str(i)+'.JPG', crop_img)

In [4]:
images_path = './Data/RBNR/set1_org/'
images = [file for file in os.listdir(images_path) if file[-3:]=='JPG']

output_path = './Data/Crops/'

In [22]:
for image in images:
    get_cropped_image(image, images_path, output_path)